In [18]:
####***********************************************************####
# Trabalho de Álgebra Linear * Processamento de Linguagem Natural #
## --------------Similaridade de Cosseno-------------------------##
####***********************************************************####

In [ ]:
# Instalar as Bibliotecas
%pip install pandas
%pip install nltk scikit-learn
%pip install matplotlib 
%pip install seaborn

In [ ]:
# Importar as Bibliotecas
import pandas as pd
import numpy as np
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Baixar stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
# Carregar dados
#url = 'https://www.kaggleusercontent.com/datasets/khushipitroda/airline-reviews/download?datasetVersionNumber=1'
# Baixar manualmente do arquivo do Kaggle e carregar localmente:
df = pd.read_csv('Airline_Reviews.csv')


In [ ]:
# Verificar colunas
print(df.columns)

In [ ]:
# Filtrar colunas necessárias
df = df[['Airline Name', 'Review', 'Overall_Rating']]
df.dropna(subset=['Review', 'Airline Name'], inplace=True)

In [ ]:
# Função de limper texto
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Remove pontuação e números
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['cleaned_review'] = df['Review'].apply(clean_text)

In [ ]:
# Vetorização com TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['cleaned_review'])

In [ ]:
# Similaridade do cosseno entre todos os pares de avaliações
cos_sim_matrix = cosine_similarity(X)


In [ ]:
# Adicionar média de similaridade por companhia aérea
df['mean_similarity'] = cos_sim_matrix.mean(axis=1)

In [ ]:
# Agrupar por companhia aérea
airline_scores = df.groupby('Airline Name').agg({
    'mean_similarity': 'mean',
    'Overall_Rating': 'mean',
    'Review': 'count'
}).reset_index()

In [ ]:
# Normalizar as pontuações para melhor visualização
scaler = MinMaxScaler()
airline_scores[['mean_similarity', 'Overall_Rating']] = scaler.fit_transform(airline_scores[['mean_similarity', 'Overall_Rating']])

In [ ]:
# Criar uma métrica combinada (peso maior para similaridade)
airline_scores['score'] = (0.7 * airline_scores['mean_similarity']) + (0.3 * airline_scores['Rating'])

In [ ]:
# Ordenar da melhor para a pior
top_airlines = airline_scores.sort_values(by='score', ascending=False)

In [ ]:
# Exibir as 5 melhores
print("Top 5 Companhias Aéreas Mais Recomendadas com Base em Análises:")
print(top_airlines[['Airline Name', 'score', 'Review']].head(5))

In [ ]:
# Visualização
plt.figure(figsize=(10, 6))
sns.barplot(data=top_airlines.head(10), x='score', y='Airline Name', palette='viridis')
plt.title('Top 10 Companhias Aéreas Mais Recomendadas (Com base em similaridade + avaliação)')
plt.xlabel('Score Normalizado')
plt.ylabel('Companhia Aérea')
plt.tight_layout()
plt.show()